In [10]:
import pandas as pd
import json
from konlpy.tag import Okt
from tqdm import tqdm
from gensim.models import Word2Vec


In [9]:
import pandas as pd
xlsx = pd.read_excel("data-files/emotion_conversation.xlsx")
xlsx.to_csv("data-files/emotion_conversation.csv")

In [15]:
# 1. 유튜브 댓글 불러오기
df_youtube = pd.read_csv("data-files/youtube_comments.csv")
youtube_comments = df_youtube['Comment'].dropna().tolist()

# 2. 네이버 영화 리뷰 불러오기
df_ratings = pd.read_csv("data-files/ratings.txt", sep='\t').dropna(subset=['document'])
movie_reviews = df_ratings['document'].tolist()

# 3. 감성대화 CSV 불러오기
df_emotion = pd.read_csv("data-files/emotion_conversation.csv")
emotion_sentences = df_emotion[['사람문장1', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3']].values.flatten().tolist()

In [21]:
youtube_comments

["0:00:01 Paul Blanco - Summer\r\n0:03:17 김승민 - 내 기쁨은 너가 벤틀리를 끄는 거야\r\n0:06:13 애쉬아일랜드 - 악몽\r\n0:08:08 TOIL - 검정색하트\r\n0:12:00 Skinny Brown, TOIL - Don't flex on me\r\n0:16:38 서동현 - vancouver\r\n0:20:12 재하 - 안았어야해",
 '외국에서 듣는 노래 너무 좋아',
 'paul Blanco',
 '밴쿠버ㄹㅈㄷ',
 'ㅡ',
 '운전 시, 매번 귀 호강.',
 '일하면서 1일1플레이 하고있습니다. 노동요로 최고임',
 '어떻게 내가 좋아하는 노래들만 담아둔거지',
 '2022넴새 난다 😢',
 '내 플리 훔쳐봤나 똑같은데...?',
 '와 플리 진짜 내 개 최애❤',
 '쌍용고등학교 이현준ㅇ ㅣㅂ.  ㄴ ㅣ다😂❤',
 '안녕하세요 저는 인의초등학교 4-2반 채린이 입니다 선플달기숙제를 하기위해 댓글을 달고 있습니다! 좋은 노래 들려주셔서 감사합니다!',
 '우연히 알게 된 채널인데, 음악에 담긴 감정과 노력이 너무 와 닿아서 감동받았어요! 저도 곡을 만들고 노래하는 채널을 운영 중인데, 시간 되시면 한 번 들러주세요. 함께 음악 이야기 나눠요! 계속 응원할게요 🎶',
 '이건 진짜 내 취향 저격💘 주인장 센스 미쳤다… 감성 터지는 힙합 최고야 🎧🌙✨',
 '너가 그때 그만할까라고 물었을때 잡았어야해 \n진짜 후회된다 너 상황을 생각도 못하고 매번 상황을 넘길려고 했던 나를 탓해줘 너가 이글을 읽었으면 좋겠어',
 '너무 감미로운 멜로디네요! 모두 건강하고 행복하세요! 🎹💖',
 '와 내기쁨은 너가 벤틀리를 끄는거야 아는사람 진짜 없는데.. 넣어주셔서 감사해요!',
 '주인장 취향 미쳤다… 노래 하나하나 다 분위기 끝판왕이네요',
 '오 제가 좋아하는 노래들 밖에 없네요 하나하나 찾기 귀찮았는데 이렇게 노래를 모아두니까 너무 좋네요 오늘도 집 갈때 들으면서 가야겟어요 감사합니다',
 '아동 범죄자 의심과 신고 

In [22]:
movie_reviews

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ',
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.',
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.',
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지',
 '안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.',
 '사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화',
 '완전 감동입니다 다시봐도 감동',
 '개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음',
 '굿',
 '바보가 아니라 병 쉰 인듯',
 '내 나이와 같은 영화를 지금 본 나는 감동적이다..하지만 훗날 다시보면대사하나하나 그 감정을완벽하게 이해할것만 같다...',
 '재밌다',
 '고질라니무 귀엽다능ㅋㅋ',
 '영화의 오페라화라고 해야할 작품. 극단적 평갈림은 어쩔 수 없는 듯.',
 '3도 반전 좋았제 ^^',
 '평점 왜 낮아? 긴장감 스릴감 진짜 최고인데 진짜 전장에서 느끼는 공포를 생생하게 전해준다.',
 '네고시에이터랑 소재만 같을 뿐.. 아무런 관련없음..',
 '단연 최고',
 '가면 갈수록 더욱 빠져드네요 밀회 화이팅!!',
 '어?생각없이 봤는데 상당한 수작.일본영화 10년내 최고로 마음에 들었다.강렬한 임팩트가 일품.',
 '오랜만에 본 제대로 된 범죄스릴러~',
 "그런 때가 있었다. ('사랑해'도 아니고) 그저 좋아한다는 한 마디 말을 꺼내기도 벅차서 밤 잠 설치던 때. 커징텅의 교복에 남은 션자이의 볼펜 자국 역시 미처 다 전하지 못한 마음의 형태인 거다.",
 '마지막씬을 잊을수가 없다',
 '강압적용서,세뇌적용서에 대한 비판',
 '중세시대명작. 굿 평점이왜이래',
 '7시간짜리 영상이 존재한다면, 죽기전에 꼭 한번 보고싶다. 아름답고 슬픈 OST

In [16]:
emotion_sentences

['이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.',
 '실수하시다니 정말 미안한 마음이 크겠어요.',
 '내 능력이 부족한 거 같은데 그만 다녀야 될거같아.',
 '능력을 올리려면 어떤 방법이 있을까요?',
 '퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.',
 '꼭 좋은 결과 있길 바라요.',
 '회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.',
 '큰 프로젝트를 혼자 하셔서 고민이 많겠네요.',
 '나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.',
 '프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?',
 '동료 직원에게 도움을 요청해서 같이 해결해야겠어.',
 '동료 직원에게 도움을 요청하기로 하셨군요.',
 '상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.',
 '직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.',
 '무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.',
 '상사분과 친해질 수 있는 방법은 무엇이 있을까요?',
 '먼저 다가가서 말을 걸어볼게.',
 '직장 상사와 친해지시면 좋겠네요.',
 '이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.',
 '첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.',
 '첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.',
 '잘 적응 하시려면 무엇을 할 수 있을까요?',
 '직장 동료와 상사들이랑 친하게 지내야겠어.',
 '직장에 잘 적응하시길 바라요.',
 '직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.',
 '직장 사람들 관계에 대해서 고민이시군요.',
 '내가 낯가림이 심해서 친해질 수 있을지 모르겠어.',
 '직장 사람들과 친해지려면 무슨 방법이 있을까요?',
 '서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.',
 '직장 사람들과 좋은 관계를 가지시길 응원해요.',
 '내가 평소에 말이 좀 험한 편인데

In [17]:
# ✅ 전체 병합
total_sentences = youtube_comments + movie_reviews + emotion_sentences

In [19]:
# 형태소 분석기 및 설정
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
tokenized_data = []

In [20]:
tqdm.pandas()
for sentence in tqdm(total_sentences):
    sentence = str(sentence).strip()
    if not sentence:
        continue
    tokens = okt.morphs(sentence, stem=True)
    filtered = [word for word in tokens if word not in stopwords and len(word) > 1 and word.isalpha()]
    if filtered:
        tokenized_data.append(filtered)

  0%|          | 0/275686 [00:00<?, ?it/s]

100%|██████████| 275686/275686 [23:39<00:00, 194.19it/s] 


In [24]:
# ✅ Word2Vec 학습
model = Word2Vec(
    sentences=tokenized_data,
    vector_size=100,
    window=5,
    min_count=3,
    workers=4,
    sg=1  # skip-gram
)

In [25]:
# ✅ 모델 저장
model.save("model/my_word2vec.model")